In [3]:
from datetime import datetime
import time
import os
from tqdm import tqdm
import pandas as pd
import re
import matplotlib.pyplot as plt

import numpy as np
import nltk
from nltk.corpus import stopwords
import pyLDAvis.gensim_models
import jieba.posseg as jp,jieba
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib
import jieba
from ark_nlp.factory.utils.conlleval import get_entity_bio
from tqdm import tqdm

F:\ML_ENVS\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [67]:
datalist = []
with open('./data/train_9.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    lines.append('\n')

    text = []
    labels = []
    label_set = set()

    for line in lines:
        if line == '\n':
            text = ''.join(text)
            entity_labels = []
            for _type, _start_idx, _end_idx in get_entity_bio(labels, id2label=None):
                entity_labels.append({
                    'start_idx': _start_idx,
                    'end_idx': _end_idx,
                    'type': _type,
                    'entity': text[_start_idx: _end_idx + 1]
                })

            if text == '':
                continue

            datalist.append({
                'text': text,
                'label': entity_labels,
                'BIO': labels
            })

            text = []
            labels = []

        elif line == '  O\n':
            text.append(' ')
            labels.append('O')
        else:
            line = line.strip('\n').split()
            if len(line) == 1:
                term = ' '
                label = line[0]
            else:
                term, label = line
            text.append(term)
            label_set.add(label.split('-')[-1])
            labels.append(label)


print(len(datalist))
df = pd.DataFrame(datalist)
df.info()
df.head(3)

36000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36000 entries, 0 to 35999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    36000 non-null  object
 1   label   36000 non-null  object
 2   BIO     36000 non-null  object
dtypes: object(3)
memory usage: 843.9+ KB


,text,label,BIO
0,先科（SAST）F1无线连接室外防水音柱壁挂式大功率定压公共广播喇叭校园播音系统户外防水音箱...,"[{'start_idx': 0, 'end_idx': 1, 'type': '1', '...","[B-1, I-1, O, B-1, I-1, I-1, I-1, O, B-3, I-3,..."
1,水粉颜料白色大瓶300ml浅灰蓝柠檬黄美术生色彩颜料灌装hys 淡黄 单瓶(满2瓶送调色刀1把),"[{'start_idx': 0, 'end_idx': 1, 'type': '4', '...","[B-4, I-4, B-4, I-4, B-16, I-16, B-13, I-13, B..."
2,全颂 智能手表腕带蓝牙防丢失学生跑步遥控拍照计步运动手环 智能手环黑色 智能手机通用款,"[{'start_idx': 0, 'end_idx': 1, 'type': '1', '...","[B-1, I-1, O, B-11, I-11, B-4, I-4, B-4, I-4, ..."


In [13]:
df['label'][0]

[{'start_idx': 0, 'end_idx': 1, 'type': '1', 'entity': '先科'},
 {'start_idx': 3, 'end_idx': 6, 'type': '1', 'entity': 'SAST'},
 {'start_idx': 8, 'end_idx': 9, 'type': '3', 'entity': 'F1'},
 {'start_idx': 10, 'end_idx': 13, 'type': '11', 'entity': '无线连接'},
 {'start_idx': 14, 'end_idx': 15, 'type': '7', 'entity': '室外'},
 {'start_idx': 16, 'end_idx': 17, 'type': '11', 'entity': '防水'},
 {'start_idx': 18, 'end_idx': 19, 'type': '4', 'entity': '音柱'},
 {'start_idx': 20, 'end_idx': 22, 'type': '13', 'entity': '壁挂式'},
 {'start_idx': 23, 'end_idx': 25, 'type': '11', 'entity': '大功率'},
 {'start_idx': 26, 'end_idx': 27, 'type': '11', 'entity': '定压'},
 {'start_idx': 28, 'end_idx': 33, 'type': '4', 'entity': '公共广播喇叭'},
 {'start_idx': 34, 'end_idx': 35, 'type': '7', 'entity': '校园'},
 {'start_idx': 36, 'end_idx': 39, 'type': '4', 'entity': '播音系统'},
 {'start_idx': 40, 'end_idx': 41, 'type': '7', 'entity': '户外'},
 {'start_idx': 42, 'end_idx': 43, 'type': '11', 'entity': '防水'},
 {'start_idx': 44, 'end_idx'

In [16]:
x = df['BIO'][0]
print(x)

['B-1', 'I-1', 'O', 'B-1', 'I-1', 'I-1', 'I-1', 'O', 'B-3', 'I-3', 'B-11', 'I-11', 'I-11', 'I-11', 'B-7', 'I-7', 'B-11', 'I-11', 'B-4', 'I-4', 'B-13', 'I-13', 'I-13', 'B-11', 'I-11', 'I-11', 'B-11', 'I-11', 'B-4', 'I-4', 'I-4', 'I-4', 'I-4', 'I-4', 'B-7', 'I-7', 'B-4', 'I-4', 'I-4', 'I-4', 'B-7', 'I-7', 'B-11', 'I-11', 'B-4', 'I-4', 'B-11', 'I-11', 'B-11', 'I-11', 'B-11', 'I-11', 'B-4', 'I-4', 'O', 'B-18', 'I-18', 'I-18', 'I-18', 'O']


In [18]:
x.index('O')

2

In [63]:
import random
def shuffle_entity(text, label, bio):
    len_text = len(text)
    # entity = list(text)
    entity = []
    entity_type = []

    for idx, info in enumerate(label):
        if idx ==0:
            tail=-1
            if info['start_idx']==0:
                if info['start_idx']-tail==1:
                    # for i in range(info['start_idx'], info['end_idx']+1):
                    entity.append(info['entity'])
                    entity_type.append(info['type'])
                elif info['start_idx']-tail>1:
                    for i in range(tail+1, info['start_idx']):
                        entity.append(text[i])
                        entity_type.append('O')
                    entity.append(info['entity'])
                    entity_type.append(info['type'])

            else:
                for i in range(0, info['start_idx']):
                    entity.append(text[i])
                    entity_type.append('O')

                entity.append(info['entity'])
                entity_type.append(info['type'])

        else:
            if info['start_idx']-tail==1:
                # for i in range(info['start_idx'], info['end_idx']+1):
                entity.append(info['entity'])
                entity_type.append(info['type'])
            elif info['start_idx']-tail>1:
                for i in range(tail+1, info['start_idx']):
                    entity.append(text[i])
                    entity_type.append('O')

                entity.append(info['entity'])
                entity_type.append(info['type'])


        tail = info['end_idx']


    if info['end_idx']<len_text-1:
        for i in range(info['end_idx']+1, len_text):
            entity.append(text[i])
            entity_type.append('O')

    # print(entity)
    # print(entity_type)


    real_entity = []
    non_entity_idx = []
    for i,b in enumerate(entity_type):
        if b!='O':
            real_entity.append((entity[i], entity_type[i]))
        elif b=='O':
            non_entity_idx.append(i)

    # 交换实体顺序
    random.shuffle(real_entity)


    res_entity = [i[0] for i in real_entity]
    res_entity_type = [i[1] for i in real_entity]

    for i in non_entity_idx:
        res_entity.insert(i, entity[i])
        res_entity_type.insert(i, 'O')


    # print(res_entity)
    # print(res_entity_type)
    assert len(entity)==len(res_entity)
    assert len(entity_type)==len(res_entity_type)


    res_text = ''.join(res_entity)
    res_label = []
    for idx, t in enumerate(res_entity_type):
        if t=='O':
            res_label.append(t)
        else:
            for i in range(len(res_entity[idx])):
                if i==0:
                    res_label.append('B-'+str(t))
                else:
                    res_label.append('I-'+str(t))
    return res_text, res_label

In [69]:
texts = []
labels = []
for i in tqdm(range(len(df))):
    t, l = shuffle_entity(text=df['text'][i],
                   label=df['label'][i],
                   bio=df['BIO'][i])
    texts.append(t)
    labels.append(l)

100%|██████████| 36000/36000 [00:03<00:00, 10701.75it/s]


In [73]:
with open('./aug_train_9_SE.txt', 'w', encoding='utf-8') as f:
    for i in tqdm(range(len(texts))):
        tt = list(texts[i])
        bb = labels[i]

        for j in range(len(tt)):
            f.write(tt[j])
            f.write(' ')
            f.write(bb[j])
            f.write('\n')

        f.write('\n')



100%|██████████| 36000/36000 [00:01<00:00, 21059.01it/s]
